In [ ]:
import grammarinator as gram
import importlib
ClibJsonGenerator = importlib.util.spec_from_file_location("ClibJsonGenerator", "grammar_files/generators/")
import os
import subprocess

# MAKE PYTHON FILE - AGNOSTIC TO INPUT

In [ ]:
START_RULE = ' --rule=json'
GRAMMAR_FILE = '/clib_fullsource_gpto4.g4'
GRAMMAR_PATH = os.path.abspath("./grammar_files/grammars") + GRAMMAR_FILE
GENERATOR_OUT_PATH = os.path.abspath('./grammar_files/generators')
SYS_PATH = ' --sys-path=' + GENERATOR_OUT_PATH
GENERATOR = ' ClibJsonGenerator.ClibJsonGenerator'
DEPTH = '20'        # default=20
OUTPUT_DIR = './grammar_files/generated_inputs/clib/'
OUTPUT_PATTERN = 'test_%d.json'
NUM_OUTPUTS = '10000'


# Setting up the grammarinator instance

- First call `grammarinator-process`
    - Creates a grammarinator generator for input creation
- Call `grammarinator-generate`
    - Creates `n` random output files according to the grammars specification
    - Stores them in `OUTPUT_DIR` named as specified in `OUTPUT_PATTERN`

## Original build
- `grammarinator-process grammars/clib_combined4_gpt4.g4 --rule=clibJson -o output/`
- `grammarinator-generate --sys-path ./output/ ClibJsonGenerator.ClibJsonGenerator --rule clibJson -d 20 -o generated_inputs/test_%d.html -n 10`

In [ ]:
# grammarinator-process
# DEBUG print('grammarinator-process '+ GRAMMAR_PATH + START_RULE + ' -o ' + GENERATOR_OUT_PATH)

generator_setup = subprocess.run('grammarinator-process '+ GRAMMAR_PATH + START_RULE + ' -o ' + GENERATOR_OUT_PATH,
                shell=True,
                executable='/bin/bash')         # Generator out filename


In [ ]:
# grammarinator-generate
# DEBUG 
print('grammarinator-generate' + SYS_PATH + GENERATOR + START_RULE +' -d ' + DEPTH + ' -o' + OUTPUT_DIR + OUTPUT_PATTERN + ' -n ' + NUM_OUTPUTS)

generate = subprocess.run('grammarinator-generate' + SYS_PATH + GENERATOR + START_RULE +
                          ' -d ' + DEPTH + ' -n ' + NUM_OUTPUTS + ' -o ' + OUTPUT_DIR + OUTPUT_PATTERN, 
                          shell=True, 
                          executable='/bin/bash')
# os.listdir(os.path.abspath(OUTPUT_DIR))

# Calling libFuzzer

- `../output_files/fuzz_manifest` contains the precompiled binary (using `--coverage`)
- The harness is `clib_harness.c` which again harnesses the `../../clib/test/fuzzing/fuzz_manifest.c`

- Original build instructions were: 
### Prepare the build.sh

- remove first lines before "Copyright"
- set environment variables for CC CXX CFLAGS CXXFLAGS OUT SRC WORK etc (as seen below in path)
- the LIB_FUZZING_ENGINE specifies the harness that "we wrote"
- fuzzing executable will appear in $OUT. There you can hand input to the harness. 

### Rebuilding the fuzzing harness

- run `github restore -p` to remove all the replaced functions and allow rebuild 
- go to clib **parent** folder 
- add environment variables
    - `CC=clang CFLAGS=--coverage CXXFLAGS=--coverage CXX=clang++ SRC=$(realpath .) OUT=$(realpath output_files) WORK=$(realpath work_files) LIB_FUZZING_ENGINE=$(realpath test_grammarinator/src/clib_harness.c) ./build.sh`